In [1]:
###### FOR PACE ICE - replace GT username below ######
%cd /home/hice1/ahanasoge6/scratch/CAI/Project
!pwd

/storage/ice1/1/8/ahanasoge6/CAI/Project
/storage/ice1/1/8/ahanasoge6/CAI/Project


In [2]:
!pip install -q -U google-generativeai
!pip install -q transformers accelerate bitsandbytes

In [3]:
import os
import json
import base64
from openai import OpenAI
import google.generativeai as genai
from dotenv import load_dotenv
from tqdm import tqdm

/home/hice1/ahanasoge6/.local/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.13) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/usr/local/pace-apps/manual/packages/anaconda3/2023.03/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# load_dotenv()
# API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_KEY = ""
GOOGLE_API_KEY = ""


In [5]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)
genai.configure(api_key=GOOGLE_API_KEY)


In [6]:
if not OPENAI_API_KEY:
    raise ValueError(" OPENAI_API_KEY not found in .env file")
if not GOOGLE_API_KEY:
    raise ValueError("Google API Key is missing")
    
print("Available models for your key:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)
    
print("Available models for your key:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

Available models for your key:
models/gemini-2.5-pro-vtea-da-csi
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemin

In [7]:
IMAGE_FOLDER = "images"           
#OUTPUT_PATH = "blip_results/captions.jsonl"
OUTPUT_PATH = "llava_results/captions.jsonl"
GPT_MODEL = "gpt-4.1-mini"
GOOGLE_MODEL = genai.GenerativeModel('models/gemini-2.0-flash')
MAX_OUTPUT = 200               

In [8]:
def encode_image(image_path):
    with open(image_path, "rb") as img:
        return base64.b64encode(img.read()).decode("utf-8")

In [9]:
def generate_questions(base64_image):
    prompt = """
You are preparing controlled experimental materials for multimodal evaluation.

Given the IMAGE (provided separately), generate the following:

----------------------------------------------------------------------
1. A correct caption
----------------------------------------------------------------------
• Must accurately describe the visible scene.
• 7–15 words, objective, simple, and factual.
• Must mention the main subject(s) and one key attribute
  (e.g., species, color, object type, action, or spatial relation).
• Should be worded well and clearly. 

----------------------------------------------------------------------
2. A deliberately incorrect caption
----------------------------------------------------------------------
• Must keep the same length and sentence structure style as the correct caption.
• MUST change EXACTLY TWO meaningful visual attributes from the correct caption.
  Allowed attribute types:
     – species/category of the main object
     – color of a main object
     – pattern/texture of a main object
     – object type that a person is holding/using
     – action the main subject is performing
     – spatial relation (e.g., “in front of” → “behind”)
     
• The incorrect caption MUST be **factually wrong for THIS image**.
  It should contradict TWO concrete visual facts visible in the picture, not merely
  describe an alternative plausible real-world scenario -- VERY IMPORTANT!
  (Example: If the scene shows a lake, “ocean” is *not* allowed because both can
   coexist conceptually; the changed attributes must be unambiguously false.)

• The incorrect caption must remain syntactically valid and plausible for the 
  kind of world the image depicts, but factually wrong.

• The two changed attributes MUST be *the most visually important attributes*
  from the correct caption.

----------------------------------------------------------------------
3. Five Visual Necessity Ladder (VNL) questions (L0–L4)
----------------------------------------------------------------------

L0 — Pure language prior  
• Must be answerable with NO access to the image.  
• General world knowledge only; do NOT reference animals, people,
  objects, nature, or environments.  
• 6–14 words.

L1 — Probe changed attribute #1 
• MUST directly probe the FIRST changed attribute from the incorrect caption.  
• Example:If species changed, ask “What type of animal…?”  
          If color changed, ask “What color is…?”  
          If object type changed, ask “What object is… holding?”  
• No attributes other than the first changed one.  
• 6–14 words.

L2 — Probe changed attribute #2
• MUST directly probe the SECOND changed attribute from the incorrect caption.  
• Same rules as L2 but targeting the second changed detail.  
• Should not be the same question as L1. 
• 6–14 words.

L3 — High-level reasoning
• Ask a reasoning question that is loosely related to the scene shown in the image.
• The question MUST NOT depend on the two changed attributes.
• The question MUST NOT target the same object/attribute as L1 or L2.
• The question SHOULD require general common-sense or contextual reasoning.
• The question SHOULD still be answerable using the image (but only its general context, not the altered details).
• 6–14 words.

----------------------------------------------------------------------
GENERAL RULES
----------------------------------------------------------------------
• Do NOT provide answers.
• Do NOT describe the image outside captions.
• All questions must be 6–14 words.
• Output MUST be a single JSON object in the exact format below.

----------------------------------------------------------------------
Return EXACTLY this JSON structure:
----------------------------------------------------------------------
{
  "correct_caption": "<string>",
  "incorrect_caption": "<string>",
  "L0": "<string>",
  "L1": "<string>",    // targets changed attribute #1
  "L2": "<string>",    // targets changed attribute #2
  "L3": "<string>"   
}


"""
    response = openai_client.responses.create(
        model=GPT_MODEL,
        max_output_tokens=MAX_OUTPUT,
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": prompt},
                    {
                        "type": "input_image",
                        "image_url": f"data:image/jpeg;base64,{base64_image}"
                    }
                ]
            }
        ]
    )

    return json.loads(response.output_text)

In [11]:
# def ask_blip2(image_path, caption, question, max_new_tokens=50):
#     """
#     Runs BLIP-2 Flan-T5-xl on:
#         IMAGE + (caption + question) text prompt.
#     Returns: the generated answer as a clean string.
#     """

#     # ---- Load image ----
#     image = Image.open(image_path).convert("RGB")

#     # ---- Build prompt for BLIP-2 ----
#     # Format:  "<caption>\n\nQuestion: <question>\nAnswer:"
#     prompt = f"{caption}\n\nQuestion: {question}\nAnswer:"

#     # ---- Preprocess ----
#     inputs = processor(
#         image,
#         prompt,
#         return_tensors="pt"
#     ).to(model.device, dtype=torch.float16)

#     # ---- Generate answer ----
#     output_ids = model.generate(
#         **inputs,
#         max_new_tokens=max_new_tokens,
#         temperature=0.0,      # deterministic & stable (recommended)
#         do_sample=False
#     )

#     # ---- Decode ----
#     answer = processor.tokenizer.decode(
#         output_ids[0],
#         skip_special_tokens=True
#     )

#     # ---- Strip the prompt part (BLIP-2 often echoes input) ----
#     if "Answer:" in answer:
#         answer = answer.split("Answer:", 1)[-1].strip()

#     # Clean spacing
#     return answer.strip()

In [12]:
def ask_llava(image_path, caption, question, max_new_tokens=50):
    """
    Runs LLaVA-1.5 on:
        IMAGE + (caption + question) text prompt.
    """
    # 1. Load Image
    image = Image.open(image_path).convert("RGB")

    # 2. Format Prompt (LLaVA specific chat format)
    # Structure: USER: <image>\nContext: {caption}\nQuestion: {question}\nAnswer the question concisely.\nASSISTANT:
    text_prompt = f"USER: <image>\nContext: {caption}\nQuestion: {question}\nAnswer the question concisely.\nASSISTANT:"

    # 3. Preprocess
    inputs = processor(
        text=text_prompt, 
        images=image, 
        return_tensors="pt"
    ).to(model.device, dtype=torch.float16)

    # 4. Generate
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False, # Deterministic for testing
            temperature=0.0
        )

    # 5. Decode
    answer = processor.decode(
        output_ids[0], 
        skip_special_tokens=True
    )

    # 6. Clean up output (Remove the input prompt from the result)
    if "ASSISTANT:" in answer:
        answer = answer.split("ASSISTANT:")[-1].strip()
        
    return answer

In [13]:
###### FOR PACE ICE ONLY - replace GT username below ######

# Tells HuggingFace to save all downloaded models + datasets in scratch directory instead of home directory
os.environ["HF_HOME"] = "/home/hice1/ahanasoge6/scratch/CAI/Project/huggingface"
os.environ["HF_DATASETS_CACHE"] = "/home/hice1/ahanasoge6/scratch/CAI/Project/hf_cache"
os.environ["TRANSFORMERS_CACHE"] = "/home/hice1/ahanasoge6/scratch/CAI/Project/hf_cache"

In [14]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
from PIL import Image

/home/hice1/ahanasoge6/.local/lib/python3.10/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [15]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# print("✅ Using device:", device)

# # "Salesforce/blip2-flan-t5-xxl" -> bigger, might need 2x memory
# MODEL_NAME = "Salesforce/blip2-flan-t5-xl"

# processor = Blip2Processor.from_pretrained(MODEL_NAME)
# model = Blip2ForConditionalGeneration.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.float16,
#     device_map="auto"
# )
# model.eval()
# print("✅ Model loaded!")

In [16]:
from transformers import AutoProcessor, LlavaForConditionalGeneration
import torch
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Using device:", device)

MODEL_NAME = "llava-hf/llava-1.5-7b-hf"

print("Loading LLaVA model...")
processor = AutoProcessor.from_pretrained(MODEL_NAME)
model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device)

print("✅ LLaVA Model loaded!")

✅ Using device: cuda
Loading LLaVA model...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.95s/it]


✅ LLaVA Model loaded!


In [17]:
def create_eval_prompt(caption, condition, question, model_answer):
    return f"""
You are an automated scoring module. You MUST output ONLY a single character: "0" or "1".
NEVER output words, sentences, explanations, punctuation, or reasoning.

YOUR TASK:
Judge whether the MODEL_ANSWER correctly matches what is visually true in the IMAGE.

IMPORTANT:
• The CAPTION was only input context for another model. Do NOT treat it as truth.
• Ignore any misleading or incorrect information in the CAPTION.
• Use ONLY the IMAGE + the QUESTION + world knowledge to judge correctness.
• If the MODEL_ANSWER matches the visible truth in the image, output "1".
• If the MODEL_ANSWER is wrong, unsupported, or contradicts the image, output "0".
• Your ENTIRE reply MUST be exactly one character: "0" or "1".

Now evaluate:

CAPTION (context only): {caption}
CONDITION: {condition}
QUESTION: {question}
MODEL_ANSWER: {model_answer}

Reply with ONLY "0" or "1".
"""

In [19]:
# def eval_answer(image_path, caption, condition, question, model_answer):
#     """
#     Builds the judge prompt, encodes the image,
#     calls Claude directly, and returns 0 or 1.
#     """

#     # ---- Build prompt ----
#     prompt = create_eval_prompt(caption, condition, question, model_answer)

#     # ---- Encode image ----
#     with open(image_path, "rb") as f:
#         img_bytes = f.read()
#     b64img = base64.b64encode(img_bytes).decode("utf-8")

#     # ---- Call Claude ----
#     response = anthropic_client.messages.create(
#         model="claude-sonnet-4-5-20250929",
#         max_tokens=5,
#         temperature=0,
#         messages=[
#             {
#                 "role": "user",
#                 "content": [
#                     {
#                         "type": "image",
#                         "source": {
#                             "type": "base64",
#                             "media_type": "image/jpeg",
#                             "data": b64img
#                         }
#                     },
#                     {
#                         "type": "text",
#                         "text": prompt
#                     }
#                 ]
#             }
#         ]
#     )

#     # ---- Parse output ----
#     output = response.content[0].text.strip()

#     if output not in ("0", "1"):
#         raise ValueError(f"Unexpected Claude judge output: {output}")

#     return int(output)

In [20]:
def eval_answer(image_path, caption, condition, question, model_answer):
    """
    Uses Google Gemini to judge the answer.
    Returns 0 or 1.
    """
    
    # 1. Load the image using PIL (Gemini likes this format)
    img = Image.open(image_path)
    
    # 2. Get your prompt text
    # (Keep your create_eval_prompt function exactly the same as it is now)
    prompt_text = create_eval_prompt(caption, condition, question, model_answer)
    
    try:
        # 3. Call Gemini
        # We pass the text prompt AND the image object in a list
        response = GOOGLE_MODEL.generate_content([prompt_text, img])
        
        # 4. Clean the output
        output = response.text.strip()
        
        # Gemini might be chatty, ensure we just get the number
        if "1" in output: return 1
        if "0" in output: return 0
        
        # Fallback if it returns something else
        return 0
        
    except Exception as e:
        print(f"Gemini Error: {e}")
        return 0

In [21]:
# SLOWER - doesn't parallelize 8 API calls 
def run():
    image_files = [
        f for f in os.listdir(IMAGE_FOLDER)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    all_results = []
    
    print(f"Found {len(image_files)} images.\n")
    
    # --- NEW: Create the output directory if it doesn't exist ---
    output_dir = os.path.dirname(OUTPUT_PATH)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        print(f"✅ Created output folder: {output_dir}")

    with open(OUTPUT_PATH, "w", encoding="utf-8") as out:
        for img_file in tqdm(image_files, desc="Processing"):
            image_id = os.path.splitext(img_file)[0]
            path = os.path.join(IMAGE_FOLDER, img_file)

            try:
                # ---- 1) GPT captions + questions ----
                b64 = encode_image(path)
                q = generate_questions(b64)

                correct_caption   = q["correct_caption"]
                incorrect_caption = q["incorrect_caption"]
#                 L0, L1, L2, L3, L4 = q["L0"], q["L1"], q["L2"], q["L3"], q["L4"]
                L0, L1, L2, L3 = q["L0"], q["L1"], q["L2"], q["L3"]


#                 # ---- 2) BLIP-2 answers ----
#                 answers_correct = {
#                     "L0": ask_blip2(path, correct_caption, L0),
#                     "L1": ask_blip2(path, correct_caption, L1),
#                     "L2": ask_blip2(path, correct_caption, L2),
#                     "L3": ask_blip2(path, correct_caption, L3),
# #                     "L4": ask_blip2(path, correct_caption, L4),
#                 }

#                 answers_incorrect = {
#                     "L0": ask_blip2(path, incorrect_caption, L0),
#                     "L1": ask_blip2(path, incorrect_caption, L1),
#                     "L2": ask_blip2(path, incorrect_caption, L2),
#                     "L3": ask_blip2(path, incorrect_caption, L3),
# #                     "L4": ask_blip2(path, incorrect_caption, L4),
#                 }

                # ---- 2) LLaVA answers ----
                answers_correct = {
                    "L0": ask_llava(path, correct_caption, L0),
                    "L1": ask_llava(path, correct_caption, L1),
                    "L2": ask_llava(path, correct_caption, L2),
                    "L3": ask_llava(path, correct_caption, L3),
                }

                answers_incorrect = {
                    "L0": ask_llava(path, incorrect_caption, L0),
                    "L1": ask_llava(path, incorrect_caption, L1),
                    "L2": ask_llava(path, incorrect_caption, L2),
                    "L3": ask_llava(path, incorrect_caption, L3),
                }

                # ---- 3) Write compact JSON ----
                output = {
                    "image_id": image_id,
                    "captions": {
                        "correct": correct_caption,
                        "incorrect": incorrect_caption
                    },
                    "questions": {
                        "L0": L0,
                        "L1": L1,
                        "L2": L2,
                        "L3": L3,
#                         "L4": L4
                    },
                    "answers": {
                        "correct": answers_correct,
                        "incorrect": answers_incorrect
                    }
                }

                out.write(json.dumps(output, ensure_ascii=False) + "\n")
                
#                 Evaluate responses with Claude-3.5 Sonnet
                for level, question in output["questions"].items():

                    # correct caption condition
                    score_c = eval_answer(
                        path,
                        output["captions"]["correct"],
                        "correct caption condition",
                        question,
                        output["answers"]["correct"][level]
                    )

                    # incorrect caption condition
                    score_i = eval_answer(
                        path,
                        output["captions"]["incorrect"],
                        "incorrect caption condition",
                        question,
                        output["answers"]["incorrect"][level]
                    )

                    all_results.append({
                        "image_id": image_id,
                        "level": level,
                        "correct_caption_score": score_c,
                        "incorrect_caption_score": score_i
                    })
                    
                print(all_results)
            

            except Exception as e:
                print(f"\n Error with {image_id}: {e}")

    print(f"\nDone. JSONL saved to: {OUTPUT_PATH}")

In [22]:
# FASTER - Parallelizes API calls
from concurrent.futures import ThreadPoolExecutor, as_completed

def run():
    image_files = [
        f for f in os.listdir(IMAGE_FOLDER)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    all_results = []
    print(f"Found {len(image_files)} images.\n")

    with open(OUTPUT_PATH, "w", encoding="utf-8") as out:
        for img_file in tqdm(image_files, desc="Processing"):
            image_id = os.path.splitext(img_file)[0]
            path = os.path.join(IMAGE_FOLDER, img_file)

            try:
                # ---- 1) GPT captions + questions ----
                b64 = encode_image(path)
                q = generate_questions(b64)

                correct_caption   = q["correct_caption"]
                incorrect_caption = q["incorrect_caption"]

                L0 = q["L0"]
                L1 = q["L1"]
                L2 = q["L2"]
                L3 = q["L3"]

                # ---- 2) BLIP-2 answers ----
#                 answers_correct = {
#                     "L0": ask_blip2(path, correct_caption, L0),
#                     "L1": ask_blip2(path, correct_caption, L1),
#                     "L2": ask_blip2(path, correct_caption, L2),
#                     "L3": ask_blip2(path, correct_caption, L3),
#                 }

#                 answers_incorrect = {
#                     "L0": ask_blip2(path, incorrect_caption, L0),
#                     "L1": ask_blip2(path, incorrect_caption, L1),
#                     "L2": ask_blip2(path, incorrect_caption, L2),
#                     "L3": ask_blip2(path, incorrect_caption, L3),
#                 }

                # ---- 2) LLaVA answers ----
                answers_correct = {
                    "L0": ask_llava(path, correct_caption, L0),
                    "L1": ask_llava(path, correct_caption, L1),
                    "L2": ask_llava(path, correct_caption, L2),
                    "L3": ask_llava(path, correct_caption, L3),
                }

                answers_incorrect = {
                    "L0": ask_llava(path, incorrect_caption, L0),
                    "L1": ask_llava(path, incorrect_caption, L1),
                    "L2": ask_llava(path, incorrect_caption, L2),
                    "L3": ask_llava(path, incorrect_caption, L3),
                }

                # ---- 3) Write compact JSON ----
                output = {
                    "image_id": image_id,
                    "captions": {
                        "correct": correct_caption,
                        "incorrect": incorrect_caption
                    },
                    "questions": {
                        "L0": L0,
                        "L1": L1,
                        "L2": L2,
                        "L3": L3
                    },
                    "answers": {
                        "correct": answers_correct,
                        "incorrect": answers_incorrect
                    }
                }

                out.write(json.dumps(output, ensure_ascii=False) + "\n")

                # ---- 4) Parallel Claude evaluation ----
                jobs = []
                #with ThreadPoolExecutor(max_workers=8) as ex:
                #Using 2 workers for gemini free tier
                with ThreadPoolExecutor(max_workers=2) as ex:
                    for level, question in output["questions"].items():

                        # correct caption condition
                        jobs.append(ex.submit(
                            eval_answer,
                            path,
                            output["captions"]["correct"],
                            "correct caption condition",
                            question,
                            output["answers"]["correct"][level]
                        ))

                        # incorrect caption condition
                        jobs.append(ex.submit(
                            eval_answer,
                            path,
                            output["captions"]["incorrect"],
                            "incorrect caption condition",
                            question,
                            output["answers"]["incorrect"][level]
                        ))

                    results = [j.result() for j in as_completed(jobs)]

                # results arrive unordered → rebuild matching pairs
                # We submitted jobs in pairs, so their order is known
                ordered_results = []
                for j in jobs:
                    ordered_results.append(j.result())

                idx = 0
                for level in ["L0", "L1", "L2", "L3"]:
                    score_c = ordered_results[idx]; idx += 1
                    score_i = ordered_results[idx]; idx += 1

                    all_results.append({
                        "image_id": image_id,
                        "level": level,
                        "correct_caption_score": score_c,
                        "incorrect_caption_score": score_i
                    })

            except Exception as e:
                print(f"\n Error with {image_id}: {e}")

    print(all_results)
    print(f"\nDone. JSONL saved to: {OUTPUT_PATH}")




In [23]:
def compute_fooling_by_level(jsonl_path):
    stats = {
        "L0": {"fooled": 0, "total": 0},
        "L1": {"fooled": 0, "total": 0},
        "L2": {"fooled": 0, "total": 0},
        "L3": {"fooled": 0, "total": 0}
    }

    with open(jsonl_path, "r") as f:
        for line in f:
            item = json.loads(line)
            eval_scores = item["eval_scores"]

            # ensure consistent level order
            for level in ["L0", "L1", "L2", "L3"]:
                scores = eval_scores[level]

                s_c = scores["correct_caption_score"]
                s_i = scores["incorrect_caption_score"]

                stats[level]["total"] += 1

                # Fooling event = correct under correct caption AND incorrect under incorrect caption
                if s_c == 1 and s_i == 0:
                    stats[level]["fooled"] += 1

    # compute fooling rates
    for lvl in stats:
        total = stats[lvl]["total"]
        fooled = stats[lvl]["fooled"]
        stats[lvl]["rate"] = fooled / total if total > 0 else 0.0

    return stats

In [24]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def run():
    image_files = [
        f for f in os.listdir(IMAGE_FOLDER)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    print(f"Found {len(image_files)} images.\n")

    with open(OUTPUT_PATH, "w", encoding="utf-8") as out:
        for img_file in tqdm(image_files, desc="Processing"):
            image_id = os.path.splitext(img_file)[0]
            path = os.path.join(IMAGE_FOLDER, img_file)

            try:
                # ---- 1) GPT captions + questions ----
                b64 = encode_image(path)
                q = generate_questions(b64)

                correct_caption = q["correct_caption"]
                incorrect_caption = q["incorrect_caption"]

                L0 = q["L0"]
                L1 = q["L1"]
                L2 = q["L2"]
                L3 = q["L3"]

                # ---- 2) BLIP-2 answers ----
#                 answers_correct = {
#                     "L0": ask_blip2(path, correct_caption, L0),
#                     "L1": ask_blip2(path, correct_caption, L1),
#                     "L2": ask_blip2(path, correct_caption, L2),
#                     "L3": ask_blip2(path, correct_caption, L3)
#                 }

#                 answers_incorrect = {
#                     "L0": ask_blip2(path, incorrect_caption, L0),
#                     "L1": ask_blip2(path, incorrect_caption, L1),
#                     "L2": ask_blip2(path, incorrect_caption, L2),
#                     "L3": ask_blip2(path, incorrect_caption, L3)
#                 }

                # ---- 2) LLaVA answers ----
                answers_correct = {
                    "L0": ask_llava(path, correct_caption, L0),
                    "L1": ask_llava(path, correct_caption, L1),
                    "L2": ask_llava(path, correct_caption, L2),
                    "L3": ask_llava(path, correct_caption, L3),
                }

                answers_incorrect = {
                    "L0": ask_llava(path, incorrect_caption, L0),
                    "L1": ask_llava(path, incorrect_caption, L1),
                    "L2": ask_llava(path, incorrect_caption, L2),
                    "L3": ask_llava(path, incorrect_caption, L3),
                }
            
                # ---- 3) Base JSON structure ----
                output = {
                    "image_id": image_id,

                    "captions": {
                        "correct": correct_caption,
                        "incorrect": incorrect_caption
                    },

                    "questions": {
                        "L0": L0,
                        "L1": L1,
                        "L2": L2,
                        "L3": L3
                    },

                    "answers": {
                        "correct_caption": answers_correct,
                        "incorrect_caption": answers_incorrect
                    },

                    "eval_scores": {}   # will be filled next
                }

                # ---- 4) Parallel Claude evaluation ----
                jobs = []
                with ThreadPoolExecutor(max_workers=8) as ex:
                    for level, question in output["questions"].items():

                        # correct caption condition
                        jobs.append(ex.submit(
                            eval_answer,
                            path,
                            output["captions"]["correct"],
                            "correct caption condition",
                            question,
                            output["answers"]["correct_caption"][level]
                        ))

                        # incorrect caption condition
                        jobs.append(ex.submit(
                            eval_answer,
                            path,
                            output["captions"]["incorrect"],
                            "incorrect caption condition",
                            question,
                            output["answers"]["incorrect_caption"][level]
                        ))

                    # collect results
                    ordered_results = [j.result() for j in jobs]

                # ---- 5) Attach scores to JSON in correct structure ----
                idx = 0
                for level in ["L0", "L1", "L2", "L3"]:
                    score_c = ordered_results[idx]; idx += 1
                    score_i = ordered_results[idx]; idx += 1

                    output["eval_scores"][level] = {
                        "correct_caption_score": score_c,
                        "incorrect_caption_score": score_i
                    }

                # ---- 6) Write one JSON line ----
                out.write(json.dumps(output, ensure_ascii=False) + "\n")

            except Exception as e:
                print(f"\nError with {image_id}: {e}")
                

    print(f"\nDone. JSONL saved to: {OUTPUT_PATH}\n")
    stats = compute_fooling_by_level(OUTPUT_PATH)
    print(stats)


In [26]:
if __name__ == "__main__":
    run()

Found 1 images.



Processing: 100%|██████████| 1/1 [00:07<00:00,  7.36s/it]


Done. JSONL saved to: llava_results/captions.jsonl

{'L0': {'fooled': 0, 'total': 1, 'rate': 0.0}, 'L1': {'fooled': 1, 'total': 1, 'rate': 1.0}, 'L2': {'fooled': 0, 'total': 1, 'rate': 0.0}, 'L3': {'fooled': 1, 'total': 1, 'rate': 1.0}}
